# SetUp

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680533 sha256=18d1852bdf42f993c54ef56cc5d5f8ce845cab1a2e0b43eb059b2faff44d68f4
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nggh9j3c
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nggh9j3c
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# Data Preprocess

In [ ]:
import pandas as pd

fpath = 'drive/MyDrive/tripbuilder_data/TLD/'

In [ ]:
data = pd.read_json(fpath+'TLD_labeled_sentence_class.json')

In [ ]:
data.count()

token           272078
label+weight    272078
label           272078
class           272078
dtype: int64

In [ ]:
print(data['class'].unique())
print(len(data['class'].unique()))

['덥다' '좋다' '싸' ... '빛나다' '취함' '녹아내리다']
1202


In [ ]:
print(data['class'].value_counts())

맛있다      86853
좋다       15053
많다        5165
맛         5116
맛집        4878
         ...  
드럽다          1
조화           1
흡연           1
카페인          1
녹아내리다        1
Name: class, Length: 1202, dtype: int64


In [ ]:
data

,token,label+weight,label,class
0,"[날, 이, 무덥다, 집, 에만, 있다, 늘어지다]","[[있다, 2.9903415442], [무덥다, 2.6313017458], [Mai...","[있다, 무덥다, Main, 덥다, 먹다, 햇볕, 겨울, 좋다, 보다, 여름날]",덥다
1,"[\r, 물론, 저희, 집, 사랑둥이, 댕댕, 이, 도, 마찬가지, 이다]","[[Main, 4.1305050254], [있다, 3.7424064875000003...","[Main, 있다, 좋다, 차다, 맛있다, 먹다, 하다, 가족, 흡족하다, 함께]",좋다
2,"[깻잎, 짱, 아지, 에도, 클래식, 쌈, 버전, 에도]","[[싸, 3.2734702975], [싸다, 2.8892546892], [먹다, 2...","[싸, 싸다, 먹다, 곁들이다, Main, 맛있다, 클래식, 밑반찬, 같이, 찰떡]",싸
3,"[그만큼, 반찬, 양, 모, 자르다, 없이, 넉넉하다, 0, 된장국, 을, 포함, ...","[[먹다, 42.4269410968], [Main, 41.1352552027], [...","[먹다, Main, 맛있다, 좋다, 하다, 있다, 보다, 같이, 음식, 먹기]",맛있다
4,"[마치, 오, 겹, 이, 아니다]","[[Main, 1.7921049893], [확실하다, 1.7247566879], [...","[Main, 확실하다, 보다, 깔보다, 장난, 굉장하다, 차다, 엄청, 먹다, 보들...",깔보다
...,...,...,...,...
272073,"[끝, 에, 앉다, 옆, 에, 뷰, 는, 산사태, 뷰, 여, 도, 바람, 이, 들어...","[[있다, 5.9701747149], [앉다, 5.8331919163], [Main...","[있다, 앉다, Main, 시원하다, 창가, 뷰, 먹다, 보다, 테라스, 야외]",앉다
272074,"[여름, 에는, 이집, 덥다]","[[덥다, 2.289532125], [여름, 2.2349573672], [겨울, 1...","[덥다, 여름, 겨울, 여름철, 시원하다, 쌀쌀하다, 계절, 습하다, 더워지다, M...",덥다
272075,"[새우, 들, 고, 사진, 찍다, 새우, 미끄러지다, 떨어지다, 순간, 의, 사진]","[[사진, 4.7731201202], [보다, 3.9424826503], [콕콕, ...","[사진, 보다, 콕콕, 찰칵찰칵, 먹다, 벌리다, 탱글탱글하다, 떨어지다, 이리저리...",사진
272076,"[5060, 대다, 어른, 들, 의, 성지인, 듯, 귀, 찜, 소자, 25000원,...","[[Main, 8.8701860905], [하다, 7.1930857599], [있다...","[Main, 하다, 있다, 가격, 보다, 먹다, 좋다, 맛있다, 차다, 흡족하다]",가격


In [ ]:
data_use = pd.DataFrame({'token': [], 'class': []})

In [ ]:
data_use

,token,class


In [ ]:
data_sent = ["".join(sent) for sent in data["token"]]
data_use['token'] = data_sent
data_use['class'] = data['class']

In [ ]:
data_use

,token,class
0,날이무덥다집에만있다늘어지다,덥다
1,\r물론저희집사랑둥이댕댕이도마찬가지이다,좋다
2,깻잎짱아지에도클래식쌈버전에도,싸
3,그만큼반찬양모자르다없이넉넉하다0된장국을포함한10가지반찬과함께탄수화물인밥을먹다간단하다...,맛있다
4,마치오겹이아니다,깔보다
...,...,...
272073,끝에앉다옆에뷰는산사태뷰여도바람이들어오다시원하다,앉다
272074,여름에는이집덥다,덥다
272075,새우들고사진찍다새우미끄러지다떨어지다순간의사진,사진
272076,5060대다어른들의성지인듯귀찜소자25000원래20000이다것으로기억하다물가상승으로인...,가격


In [ ]:
#train & test 데이터로 나누기
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(list(data_use['class']))

encoded_words = encoder.transform(data_use["class"])
word_dict = dict(zip(range(len(encoder.classes_)), encoder.classes_))

data_new_use = pd.DataFrame({'sentence': data_use['token'], 'class': encoded_words})

In [ ]:
data_new_use

,sentence,class
0,날이무덥다집에만있다늘어지다,193
1,\r물론저희집사랑둥이댕댕이도마찬가지이다,878
2,깻잎짱아지에도클래식쌈버전에도,587
3,그만큼반찬양모자르다없이넉넉하다0된장국을포함한10가지반찬과함께탄수화물인밥을먹다간단하다...,265
4,마치오겹이아니다,98
...,...,...
272073,끝에앉다옆에뷰는산사태뷰여도바람이들어오다시원하다,624
272074,여름에는이집덥다,193
272075,새우들고사진찍다새우미끄러지다떨어지다순간의사진,416
272076,5060대다어른들의성지인듯귀찜소자25000원래20000이다것으로기억하다물가상승으로인...,7


In [ ]:
data_split = data_new_use.to_numpy().tolist()

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_split, test_size=0.15, shuffle=True, random_state=34)

In [ ]:
dataset_train

[['돼지고기도잡다없이깔끔하다식감이쫀득하다', 175],
 ['사진에보이다게2인분이다', 416],
 ['이벤트참여하고계란찜꼭들다먹다보다준비되다고기김치짜다글이스톱워치를준비해주다갈다', 265],
 ['사장님이술을담그다걸장에보관하다서다사람마다한참보다장뒷편으로가면화장실이다안쪽까지자리가제법있다손님들면치기하는걸보니까더배고프다있다고문이이다',
  878],
 ['옛날얘기에신나다30대다1예전에는유가네닭갈비에서닭갈비나철판볶음밥메뉴를주문하다테이블위에서휙휙멋지다손놀림으로볶다포포몬쓰다유가네닭갈비가인기를많이얻다되다기도한상징이다것같다이제는메뉴를주문하다주방에서다볶다나오다깔끔하다시스템으로또바뀌다솔직하다테이블위에서볶다보다재미는있다기름이상당하다많이튀다것으로기억이돼다',
  265],
 ['한마리8천원두마리15천원', 165],
 ['부산가족외식부산쪽갈비명지쪽갈비강서구쪽갈비강서구밥집명지밥집명지고기집명지맛집강서구맛집명지음식점명지외식강서구외식부산외식부산맛집부산음식점강서구음식점부산가족외식부산쪽갈비명지쪽갈비강서구쪽갈비강서구밥집명지밥집명지고기집명지맛집강서구맛집명지음식점명지외식강서구외식부산외식부산맛집부산음식점강서구음식점',
  266],
 ['추다날따끈하다국물요리와소주한잔해보다하다옛날부터유명하다단골이많다케이닥으로가보다외관은이렇게포차처럼되어다옛날에지나가다보다때는진짜비닐포차처럼되어다지금은건물을세우다튼튼하다간판도제작되다그만큼유명하다맛있다다들다찾다거겠다날어어어엄청춥다',
  265],
 ['육수만붓다끓이다끝냄비에옮기다담다과정조차없다너무편하다밀키트도뭔가손이가다조리법은하다귀찮다정말귀차니즘폭발한날에딱이다메뉴이다', 265],
 ['이제는가마치콤보후라이드반양념반', 1102],
 ['부드럽다고기도꽤들다', 360],
 ['라멘을좋아하다이번에는서면에생기다얼마안되다가다가보다', 431],
 ['알감자튀김은상당하다짜다펴다호불호가심하다갈릴듯한맛이이다', 1156],
 ['결국그나마가깝다경주에가기로애기들데리고너무멀리오다가다힘들다같다예약한곳인데찍다너무마음에들다가족사진찍다갈다예쁘다입다낮잠시간맞추다출발요즘신랑한테서운하다일폭발이라

In [ ]:
dataset_test

[['이웃님들다후라이드VS양념어떻다걸선호하다저는후라이드파양념소스추가로받다콕찍다먹다걸좋아하다기본반찬치킨무케요네즈양배추샐러드개인적으로케쳡과마요네즈를싫어하다샐러드한입도안먹다',
  265],
 ['기분좋다우리가상치우다앉다부리다요새이렇다가격찾기쉽지않쥬감사하물다본안주금방나오다육사시미신선하다좋다소스도여러가지코찡까지어머이탕슉은사장님서비스이다뭔가다른거먹다사장님이게무슨일',
  265],
 ['저는알쓰다그냥닭하다마리를테이크아웃하다주문하다', 1146],
 ['전3만원대로가성비좋다즐기다수있다메뉴인통뼈닭발옛날통닭세트를시키다', 897],
 ['과연그맛은어떻다한양각동래중국집동래중식당동래구맛집낙민동맛집부산동래구맛집부산짬뽕맛집부산짜장면맛집동래한양아파트부산동래중국집한양각동래중국집동래중식당동래구맛집낙민동맛집부산동래구맛집부산짬뽕맛집부산짜장면맛집동래한양아파트부산동래중국집가게는동래한양아파트앞상가2층에있다',
  266],
 ['다들다배고프다완전맛있다흡입하고오늘의일정경주엑스포공원으로고고식당에서5분거리임', 265],
 ['시골추어탕시골추어탕그릇에담다마늘다데기와다지다고추그리고꼭들어가다하다산초가루를넣다숟가락으로휙휙저어서들다그맛이환상이다', 265],
 ['40년전통으로부산대신동맛집으로자리잡다곳이다', 266],
 ['며칠전지인의축일이라점식식사를하다가다다가가다식당이정기휴일이라예전의기억을더듬다찾아가다곳이다', 954],
 ['그때그시절포장마차분식을찾다분2', 1091],
 ['술못하다오빠도이날만큼은같이속상하다하다두잔해주다하지만두잔딱먹다취하다버리다우리남편', 28],
 ['근데그래도웬만하다서동치킨가게는이값에못비비다걸요즘하다마리에15000원은기본인데부산가마치통닭은많이오르다두마리에15000원이니까장난아니다',
  265],
 ['북쪽은낙동강을경계로밀양시양산시과접하고동쪽은낙동강건너양산시부산광역시강서구남쪽은창원시진해구서쪽은창원시의창구성산구에접하고있다', 5],
 ['그렇다고점심이나저녁을먹다않다없다들어가다음식을먹다되다어떤땐무심코들어가다맛집수준의음식을먹다되다경우도있다블로거들이\xa0맛집이라고올리다집을\xa0가다이것

In [ ]:
len(data_new_use['class'].unique())

1202

# Model-setting

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 30
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=1202,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# Training & Testing

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-30-ee26ec442099>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/14455 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 7.104491710662842 train acc 0.0
epoch 1 batch id 201 loss 6.943164348602295 train acc 0.026119402985074626
epoch 1 batch id 401 loss 5.979205131530762 train acc 0.1630299251870324
epoch 1 batch id 601 loss 4.633918285369873 train acc 0.2197379367720466
epoch 1 batch id 801 loss 4.997292518615723 train acc 0.24516229712858925
epoch 1 batch id 1001 loss 4.966741561889648 train acc 0.2603646353646354
epoch 1 batch id 1201 loss 4.092983245849609 train acc 0.2716486261448793
epoch 1 batch id 1401 loss 4.826812744140625 train acc 0.2766773733047823
epoch 1 batch id 1601 loss 4.495662212371826 train acc 0.28306527170518425
epoch 1 batch id 1801 loss 4.916621685028076 train acc 0.28758328706274294
epoch 1 batch id 2001 loss 3.5028693675994873 train acc 0.29079210394802596
epoch 1 batch id 2201 loss 4.498436450958252 train acc 0.294127669241254
epoch 1 batch id 2401 loss 2.5619843006134033 train acc 0.2969856309870887
epoch 1 batch id 2601 loss 4.574082374572754 train ac

<ipython-input-30-ee26ec442099>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/2551 [00:00<?, ?it/s]

...

epoch 4 validation acc 0.917891676466745


  0%|          | 0/14455 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.012378576211631298 train acc 1.0
epoch 5 batch id 201 loss 1.3289576768875122 train acc 0.9567786069651741
epoch 5 batch id 401 loss 0.08476907759904861 train acc 0.9585411471321695
epoch 5 batch id 601 loss 0.0114755779504776 train acc 0.959234608985025
epoch 5 batch id 801 loss 0.38558509945869446 train acc 0.959503745318352
epoch 5 batch id 1001 loss 0.6336273550987244 train acc 0.9596653346653347
epoch 5 batch id 1201 loss 0.07708752155303955 train acc 0.9596169858451291
epoch 5 batch id 1401 loss 0.04108615219593048 train acc 0.9588240542469665
epoch 5 batch id 1601 loss 0.031665313988924026 train acc 0.9593222985633979
epoch 5 batch id 1801 loss 0.27603718638420105 train acc 0.9593975569128262
epoch 5 batch id 2001 loss 0.003434313926845789 train acc 0.9594265367316341
epoch 5 batch id 2201 loss 0.6894497871398926 train acc 0.959137891867333
epoch 5 batch id 2401 loss 0.001562396646477282 train acc 0.958741149521033
epoch 5 batch id 2601 loss 0.228186696

  0%|          | 0/2551 [00:00<?, ?it/s]

epoch 5 validation acc 0.9221547105710178


# Model Save

In [ ]:
torch.save(model, "drive/MyDrive/tripbuilder_modules/TRIFIT/BertClassifier_model")